In [1]:
import requests
from bs4 import BeautifulSoup
import re
from jproperties import Properties
import json
import pandas as pd

In [2]:
configs = Properties()
with open('application.properties', 'rb') as config_file:
    configs.load(config_file)

In [3]:
dotabuff_filters = configs.get("dotabuff_filters").data
dis_pkl_file_path = configs.get("json_file_path").data
wr_pkl_file_path = configs.get("json_file_path").data

In [4]:
url1 = "https://www.dotabuff.com/heroes"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

req = requests.get(url1, headers=headers)
soup = BeautifulSoup(req.content, "html.parser")
hero_links = soup.find_all("a", href=re.compile(r"/heroes/.*"))
heroes=[]
for link in hero_links:
    if link.find("div", class_="hero"):
        heroes.append({"hero_name": link.text.strip(), "link_name": link['href'][8:]})

In [25]:
hero_names_list = [hero_dict['hero_name'] for hero_dict in heroes]

print(json.dumps(hero_names_list))

["Abaddon", "Alchemist", "Ancient Apparition", "Anti-Mage", "Arc Warden", "Axe", "Bane", "Batrider", "Beastmaster", "Bloodseeker", "Bounty Hunter", "Brewmaster", "Bristleback", "Broodmother", "Centaur Warrunner", "Chaos Knight", "Chen", "Clinkz", "Clockwerk", "Crystal Maiden", "Dark Seer", "Dark Willow", "Dawnbreaker", "Dazzle", "Death Prophet", "Disruptor", "Doom", "Dragon Knight", "Drow Ranger", "Earth Spirit", "Earthshaker", "Elder Titan", "Ember Spirit", "Enchantress", "Enigma", "Faceless Void", "Grimstroke", "Gyrocopter", "Hoodwink", "Huskar", "Invoker", "Io", "Jakiro", "Juggernaut", "Keeper of the Light", "Kunkka", "Legion Commander", "Leshrac", "Lich", "Lifestealer", "Lina", "Lion", "Lone Druid", "Luna", "Lycan", "Magnus", "Marci", "Mars", "Medusa", "Meepo", "Mirana", "Monkey King", "Morphling", "Naga Siren", "Nature's Prophet", "Necrophos", "Night Stalker", "Nyx Assassin", "Ogre Magi", "Omniknight", "Oracle", "Outworld Destroyer", "Pangolier", "Phantom Assassin", "Phantom Lance

In [8]:
data_dict_dis={}
data_dict_wr={}
for hero in heroes:
    single_hero_counter_data=[]
    url2 = f"https://www.dotabuff.com/heroes/{hero['link_name']}/counters{dotabuff_filters}"
    req = requests.get(url2, headers=headers)
    soup = BeautifulSoup(req.content, "html.parser")
    header = soup.find("header", string="Matchups")
    table = header.find_next_sibling("article").find("table")
    for row in table.find_all("tr"):
        counter_name_tag = row.find("a", class_="link-type-hero")
        if row.find("td") and counter_name_tag:
            disadvantage = row.find_all("td")[2]['data-value']
            wr = row.find_all("td")[3]['data-value']
            data_dict_dis.setdefault(counter_name_tag.text.strip(), {})[hero['hero_name']] = float(disadvantage)
            data_dict_wr.setdefault(counter_name_tag.text.strip(), {})[hero['hero_name']] = float(wr)
    print(hero['hero_name'])

Abaddon
Alchemist
Ancient Apparition
Anti-Mage
Arc Warden
Axe
Bane
Batrider
Beastmaster
Bloodseeker
Bounty Hunter
Brewmaster
Bristleback
Broodmother
Centaur Warrunner
Chaos Knight
Chen
Clinkz
Clockwerk
Crystal Maiden
Dark Seer
Dark Willow
Dawnbreaker
Dazzle
Death Prophet
Disruptor
Doom
Dragon Knight
Drow Ranger
Earth Spirit
Earthshaker
Elder Titan
Ember Spirit
Enchantress
Enigma
Faceless Void
Grimstroke
Gyrocopter
Hoodwink
Huskar
Invoker
Io
Jakiro
Juggernaut
Keeper of the Light
Kunkka
Legion Commander
Leshrac
Lich
Lifestealer
Lina
Lion
Lone Druid
Luna
Lycan
Magnus
Marci
Mars
Medusa
Meepo
Mirana
Monkey King
Morphling
Naga Siren
Nature's Prophet
Necrophos
Night Stalker
Nyx Assassin
Ogre Magi
Omniknight
Oracle
Outworld Destroyer
Pangolier
Phantom Assassin
Phantom Lancer
Phoenix
Primal Beast
Puck
Pudge
Pugna
Queen of Pain
Razor
Riki
Rubick
Sand King
Shadow Demon
Shadow Fiend
Shadow Shaman
Silencer
Skywrath Mage
Slardar
Slark
Snapfire
Sniper
Spectre
Spirit Breaker
Storm Spirit
Sven
Techies


In [12]:
df_dis = pd.DataFrame.from_dict(data_dict_dis, orient='index')
df_wr = pd.DataFrame.from_dict(data_dict_wr, orient='index')
df_dis = df_dis.fillna(0)
df_wr = df_wr.fillna(50)

In [13]:
df_dis

,Abaddon,Alchemist,Ancient Apparition,Anti-Mage,Arc Warden,Axe,Bane,Batrider,Beastmaster,Bloodseeker,...,Visage,Void Spirit,Warlock,Weaver,Windranger,Winter Wyvern,Witch Doctor,Wraith King,Zeus,Slark
Slark,2.8254,1.7413,-1.6897,1.6884,2.1651,-1.1325,1.5383,3.4577,1.4067,-2.9636,...,-0.3305,-0.2107,1.9456,0.6784,-1.3419,1.5461,0.6941,1.8337,-0.5176,0.0000
Dark Seer,2.7767,0.2516,0.5553,-0.5337,0.8451,0.3188,0.0470,-1.6942,0.9832,-1.7333,...,1.0745,-0.3823,-1.2343,-1.5041,-1.7246,-0.4092,-0.7802,2.2101,0.1520,-0.3378
Undying,2.4050,-0.4893,-1.5594,-0.8089,0.5157,0.1074,0.4344,0.0499,1.6281,-0.4919,...,1.2762,0.0467,0.9454,-1.3306,-0.9802,1.5433,-0.4273,3.2598,0.5794,-0.9746
Anti-Mage,2.3031,0.6161,3.8119,0.0000,4.3435,-2.0063,0.1308,1.2391,-0.7978,-2.6377,...,-0.0693,1.7333,-0.0944,-1.0858,-0.7389,3.4808,-0.3256,2.5835,4.0419,-1.6247
Shadow Demon,2.2255,-0.2539,1.0322,-0.0591,0.3710,1.4027,0.0716,1.2646,-1.1906,0.0510,...,0.0170,-0.8862,-0.2511,-2.1801,1.7193,0.0964,1.0605,0.4542,0.5360,1.0082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Arc Warden,-3.2277,-0.8177,2.9252,-4.2297,0.0000,1.7211,0.1969,-2.0315,-0.8604,0.5893,...,-3.4136,-0.9354,-2.2090,-0.2562,0.5503,1.3569,-0.6009,-1.7877,0.0671,-2.2631
Batrider,-3.2961,0.5810,0.0255,-0.4531,2.3342,3.5887,-1.2689,0.0000,0.1351,-0.3913,...,0.9286,-0.2820,0.5778,-2.6953,-0.9598,0.7257,0.0075,0.5171,1.1846,-1.8910
Silencer,-3.4837,-1.1781,0.9251,0.4892,-1.8081,1.6979,1.8906,0.5655,-0.4622,-0.7036,...,0.4029,0.8143,-0.6121,1.3166,-0.6048,-0.1266,0.6990,-1.4601,-0.4258,-1.1765
Spectre,-3.8793,-3.1762,2.1463,-3.7320,1.1423,2.9485,0.4026,-1.6696,-2.1269,0.9170,...,-2.8217,0.1320,-3.3547,0.2111,2.5345,1.7449,-0.9412,-0.4458,1.5041,-1.6989


In [14]:
df_wr

,Abaddon,Alchemist,Ancient Apparition,Anti-Mage,Arc Warden,Axe,Bane,Batrider,Beastmaster,Bloodseeker,...,Visage,Void Spirit,Warlock,Weaver,Windranger,Winter Wyvern,Witch Doctor,Wraith King,Zeus,Slark
Slark,47.0525,43.3622,52.1462,48.8510,50.8808,52.1030,44.7784,39.2342,45.5201,50.3046,...,52.8309,47.1482,47.9450,46.5258,48.1738,45.3611,49.6134,46.8087,49.7228,50.0000
Dark Seer,47.8189,45.6678,50.9339,52.0672,53.0328,51.6893,47.0822,45.0654,46.7644,50.2792,...,52.3752,48.2897,51.9801,49.6634,49.6883,48.1683,52.2948,47.2572,50.0983,51.4268
Undying,45.9036,43.9785,50.5311,49.7001,50.9330,49.2132,44.3384,40.9501,43.8429,46.5338,...,49.8923,45.3853,47.3766,46.9373,46.2222,43.8347,49.0056,43.8049,46.9234,49.3215
Anti-Mage,47.0244,43.9486,46.3574,50.0000,48.2824,52.5136,45.6425,40.8235,47.1500,49.6496,...,52.1470,44.8014,49.4165,47.7616,47.1022,43.0081,50.1275,45.5584,44.5052,51.1488
Shadow Demon,53.0843,50.9211,55.3461,56.7077,58.3079,55.7904,51.7182,46.7538,53.5371,53.2789,...,58.0438,53.6434,55.7658,55.2059,51.2202,52.4381,55.8460,54.0179,55.0872,55.3189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Arc Warden,50.0030,42.9163,44.5980,51.7179,50.0000,45.9229,43.1676,41.6008,44.8583,44.0440,...,53.0563,44.9002,49.0672,44.4640,43.1335,42.5455,47.5154,47.3605,45.8387,49.1208
Batrider,60.6422,52.0960,58.4473,59.1756,58.4005,55.7018,55.0396,50.0000,54.1765,55.7579,...,59.1159,55.0920,56.9485,57.7868,56.2361,53.8281,59.2642,56.0291,56.6460,60.7657
Silencer,49.5272,42.6476,46.1042,46.2863,51.1455,45.4657,41.0547,38.5138,43.9006,44.7255,...,48.7735,42.6477,46.8902,42.3823,43.6880,43.4695,45.5082,46.3729,45.7049,47.2642
Spectre,50.8085,45.4265,45.6672,51.3891,49.1073,44.9792,43.2010,41.4542,46.2864,43.9595,...,52.6436,44.1119,50.3816,44.2382,41.3551,42.4354,48.2054,46.2280,44.5731,48.6991


In [15]:
import pickle
with open('dbuff_dis_data.pkl', 'wb') as f:
    pickle.dump(df_dis, f)
with open('dbuff_wr_data.pkl', 'wb') as f:
    pickle.dump(df_wr, f)